In [2]:
# installs
# !pip install bertopic 

# imports
import pandas as pd
import os
from bertopic import BERTopic
import nltk
from nltk.corpus import stopwords
# from bertopic import preprocess_text
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from tokenizers import pre_tokenizers
# nltk.download('stopwords')
    
print("imports compelete")


/data/user/anna19/Conda_Env/nlp2023v2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


imports compelete


In [5]:
## Download and decompress data

# !wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFilesSmall/Books_5.json.gz
# !wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFilesSmall/Home_and_Kitchen_5.json.gz
# !wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFilesSmall/Sports_and_Outdoors_5.json.gz
# !wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFilesSmall/Electronics_5.json.gz

import gzip
chunk_size=5 * 1024 * 1024

with gzip.open('Books_5.json.gz') as f:
    with open("Books_5.json", 'wb') as f_out:
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break
            f_out.write(chunk)
print("books done")

with gzip.open('Home_and_Kitchen_5.json.gz') as f:
    with open("Home_and_Kitchen_5.json", 'wb') as f_out:
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break
            f_out.write(chunk)
print("kitchen done")

with gzip.open('Sports_and_Outdoors_5.json.gz') as f:
    with open("Sports_and_Outdoors_5.json", 'wb') as f_out:
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break
            f_out.write(chunk)
print("sports done")

!wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFilesSmall/Electronics_5.json.gz

with gzip.open("Electronics_5.json.gz", 'rb') as f:
    with open("Electronics_5.json", 'wb') as f_out:
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break
            f_out.write(chunk)
print("electronics done")

print("done")

electronics done
done


In [1]:
%%time

# store data in lists

import json

review_text = []

training_data_books = []
training_data_outdoors = []
training_data_electronics = []
training_data_kitchen=[]
# with open('Books_5.json') as f:
#     for review in f:
#         text = json.loads(review).get("reviewText", "").strip()
#         summary = json.loads(review).get("summary", "").strip()
#         review = summary + " " + text
#         if review.strip():
#             training_data_books.append(review)
# print("Books done")
    
# with open('Home_and_Kitchen_5.json') as f:
#     for review in f:
#         review = json.loads(review)
#         text = review.get("reviewText", "").strip()
#         summary = review.get("summary", "").strip()
#         review = summary + " " + text
#         if review.strip():
#             training_data_kitchen.append(review)
            
# print("kitchen done")

with open('Sports_and_Outdoors_5.json') as f:
    for review in f:
        review = json.loads(review)
        text = review.get("reviewText", "").strip()
        summary = review.get("summary", "").strip()
        review = summary + " " + text
        if review.strip():
            training_data_outdoors.append(review)
            
print("outdoors done")

# with open('Electronics_5.json') as f:
#     for review in f:
#         review = json.loads(review)
#         text = review.get("reviewText", "").strip()
#         summary = review.get("summary", "").strip()
#         review = summary + " " + text
#         if review.strip():
#             training_data_electronics.append(review)
# print("electronics done")

outdoors done
CPU times: user 13.6 s, sys: 1.41 s, total: 15 s
Wall time: 15 s


In [1]:
%%time


UsageError: %%time is a cell magic, but the cell body is empty. Did you mean the line magic %time (single %)?


done


In [18]:
# take 80% of each dataset for training

complete_test=[]
# complete_test.extend(training_data_books[:int(0.8*len(training_data_books))])
# complete_test.extend(training_data_electronics[:int(0.8*len(training_data_electronics))])
complete_test.extend(training_data_outdoors[:int(0.8*len(training_data_outdoors))])
# complete_test.extend(training_data_kitchen[:int(0.8*len(training_data_kitchen))])
    
complete_test=complete_test[:500000]
            
print("data read in")
print(len(complete_test))
# print(len(training_data_outdoors))
print(complete_test[0])

data read in
500000
Five Stars What a spectacular tutu! Very slimming.


In [19]:
%%time
import torch
from sentence_transformers import SentenceTransformer, util

# Check if a GPU is available
if torch.cuda.is_available():
    device = 'cuda'  # You can specify the GPU device index
    print("using gpu")
else:
    device = 'cpu'

# Load a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2').to(device)


# Compute embeddings using GPU
embeddings = model.encode(complete_test, device=device, show_progress_bar=True)


print("embeddings done")



using gpu


Batches: 100%|██████████| 15625/15625 [03:25<00:00, 76.13it/s] 


embeddings done
CPU times: user 3min 17s, sys: 18.6 s, total: 3min 35s
Wall time: 3min 35s


In [20]:
%%time
# train and save model
# reference: https://maartengr.github.io/BERTopic/getting_started/best_practices/best_practices.html
# reference: https://medium.com/rapids-ai/faster-topic-modeling-with-bertopic-and-rapids-cuml-5c7559aba898
# https://towardsdatascience.com/topic-modeling-with-lsa-plsa-lda-nmf-bertopic-top2vec-a-comparison-5e6ce4b1e4a5

print("process begun")
print(f"training data amount: {len(complete_test)}")
# !pip install \
#     --extra-index-url=https://pypi.nvidia.com \
#     cudf-cu12 dask-cudf-cu12 cuml-cu12 cugraph-cu12 cuspatial-cu12 cuproj-cu12 cuxfilter-cu12 cucim
# !pip uninstall cupy-cuda115
# !pip install cupy-cuda11x
import os
from sklearn.feature_extraction.text import CountVectorizer
# from cuml.cluster import HDBSCAN
# from cuml.manifold import UMAP
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic import BERTopic
# import openai
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, OpenAI, PartOfSpeech

umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

hdbscan_model = HDBSCAN(min_cluster_size=150, metric='euclidean', cluster_selection_method='eom', prediction_data=True)


os.environ['TOKENIZERS_PARALLELISM'] = 'false'
print(f"training data amount: {len(complete_test)}")


# we add this to remove stopwords
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english", min_df = 2)


# KeyBERT
keybert_model = KeyBERTInspired()

# Part-of-Speech
pos_model = PartOfSpeech("en_core_web_sm")

# MMR
mmr_model = MaximalMarginalRelevance(diversity=0.3)

# GPT-3.5
# openai.api_key = "sk-..."
# prompt = """
# I have a topic that contains the following documents: 
# [DOCUMENTS]
# The topic is described by the following keywords: [KEYWORDS]

# Based on the information above, extract a short but highly descriptive topic label of at most 5 words. Make sure it is in the following format:
# topic: <topic label>
# """
# openai_model = OpenAI(model="gpt-3.5-turbo", exponential_backoff=True, chat=True, prompt=prompt)

# All representation models
representation_model = {
    "KeyBERT": keybert_model,
#     "OpenAI": openai_model,  # Uncomment if you will use OpenAI
    "MMR": mmr_model,
    "POS": pos_model
}

bert_model = BERTopic(

  # Pipeline models
  embedding_model=model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  representation_model=representation_model,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)
print("model created")

# model = BERTopic(
#     vectorizer_model=vectorizer_model,
#     hdbscan_model=cluster_model,
#     language='english', calculate_probabilities=True,
#     verbose=True
# )

topics, probs = bert_model.fit_transform(complete_test, embeddings)

bert_model.save("bertopic_model_outdoors_500K")

print("model done")

process begun
training data amount: 500000
training data amount: 500000
model created


2023-11-21 21:11:36,711 - BERTopic - Reduced dimensionality
2023-11-21 21:12:38,126 - BERTopic - Clustered reduced embeddings
/data/user/anna19/Conda_Env/nlp2023v2/lib/python3.10/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



model done
CPU times: user 9h 56min 11s, sys: 4min 45s, total: 10h 56s
Wall time: 1h 26min 28s


In [ ]:
# visualize results
print("running again")

loaded_model = BERTopic.load("bertopic_model_outdoors_500K")

loaded_model.visualize_topics().write_html("./intertopic_dist_map_outdoors_500K.html")

loaded_model.visualize_barchart(top_n_topics = 25).write_html("./barchart_outdoors_500K.html")

# # Save documents projection as HTML file
loaded_model.visualize_documents(complete_test).write_html("./projections_outdoors_500K.html")

# # Save topics dendrogram as HTML file
loaded_model.visualize_hierarchy().write_html("./hieararchy_outdoors_500K.html")

print("done")

running again


In [12]:
print(bert_model.get_topic_info()[:11])
top_words = bert_model.get_topics()

# Print the top words for each topic
for topic_id, words in top_words.items():
    words = [word[0] for word in words]
    print(f"Topic {topic_id}: {', '.join(words)}")

# # Assign topics to documents
# for i, topic in enumerate(topics):
#     print(f"Document {i + 1} is assigned to Topic {topic}")

    Topic  Count                                  Name  \
0      -1   1848              -1_great_board_good_like   
1       0   2109  0_stars_stars great_stars good_great   
2       1   1097          1_bands_resistance_band_door   
3       2    894          2_great_good_quality_product   
4       3    486                3_ball_soccer_net_goal   
5       4    445         4_stove_coleman_camping_flame   
6       5    442            5_scooter_razor_wheels_old   
7       6    353              6_frisbee_throw_fun_ring   
8       7    349           7_mattress_air_camping_pump   
9       8    339          8_compass_bike_suunto_mirror   
10      9    337             9_knife_blade_knives_tool   

                                       Representation  \
0   [great, board, good, like, just, use, water, s...   
1   [stars, stars great, stars good, great, great ...   
2   [bands, resistance, band, door, use, great, se...   
3   [great, good, quality, product, price, love, w...   
4   [ball, soccer,